In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import torch 
import torch.nn as nn
import torch.nn.functional as F
import nltk
from nltk import word_tokenize
from nltk.probability import FreqDist
from langdetect import detect
import json
import re
from collections import Counter, defaultdict
import string

In [4]:
#Cleaning out the original test and train datasets 
def langFilter(file,type:str):
    num = 0
    with open(f"hotel_review_en_{type}.JSON", "w") as out_file:
        with open(file,"r") as in_file:
            lines = in_file.readlines()
            for line in lines:
                num += 1
                # if num%100==0: print(num)
                json_line = json.loads(line)
                try:
                    lang = detect(json_line["text"])
                    if lang == "en":
                        out_file.write(line)
                except:
                    print("exception")
                    pass

#uncomment to generate JSON train and test sets of only english texts
langFilter("hotel_reviews_train.JSON","train")
langFilter("hotel_reviews_test.JSON","test")

In [11]:
train = pd.read_json('hotel_review_en_train.JSON', lines=True)
test = pd.read_json('hotel_review_en_test.JSON', lines=True)
train.head()

,ratings,title,text,author,date_stayed,offering_id,num_helpful_votes,date,id,via_mobile
0,"{'service': 1.0, 'cleanliness': 2.0, 'overall'...",“Horrible experience”,First of all we got there and they didn't have...,"{'username': 'Kh3RD', 'num_reviews': 1, 'id': ...",September 2012,80138,0,2012-09-19,140716137,False
1,"{'cleanliness': 1.0, 'sleep_quality': 1.0, 'ov...",“Stay Away”,Found Racist graffiti in the room. Request to ...,"{'username': 'TheUglyPhotographer', 'num_citie...",June 2011,240151,1,2011-06-27,114807323,False
2,"{'service': 4.0, 'cleanliness': 5.0, 'overall'...",“Great location and service”,Close to Union Square - hotel is a bit of a ma...,"{'username': 'Moonstonemoclips', 'num_cities':...",October 2010,80793,0,2010-10-25,84805430,False
3,"{'service': 3.0, 'cleanliness': 2.0, 'overall'...",“I will never go back here again!”,I had a bad vibe about this place from the mom...,"{'username': 'JoanellenJ', 'num_cities': 10, '...",June 2012,111418,1,2012-06-28,132971117,False
4,{'overall': 1.0},“Ripped off our VISA card after departure”,"After we received our ""final"" bill and left th...","{'username': 'Lynnworks', 'num_cities': 3, 'nu...",January 2012,671150,3,2012-02-04,124104157,False


In [12]:
test.head()

,ratings,title,text,author,date_stayed,offering_id,num_helpful_votes,date,id,via_mobile
0,"{'service': 5.0, 'cleanliness': 5.0, 'overall'...",“I was SO surprised! I WILL return!”,My husband and I just celebrated our 25th wedd...,"{'username': 'shooflyfarm', 'num_cities': 15, ...",November 2011,1783324,1,2011-11-26,121063682,False
1,"{'cleanliness': 5.0, 'sleep_quality': 5.0, 'ov...",“A Mother/Daughter vacation”,I could not leave a bad comment on any part of...,"{'username': 'bestmpm', 'num_reviews': 1, 'id'...",August 2011,88458,0,2011-08-10,116545869,False
2,"{'service': 5.0, 'cleanliness': 5.0, 'overall'...",“Good Choice for Pre-cruise”,I spent one night at this hotel prior to a cru...,"{'username': 'Conner2dood', 'num_cities': 20, ...",November 2010,82868,0,2010-12-06,89196759,False
3,"{'cleanliness': 5.0, 'sleep_quality': 5.0, 'ov...",“Unsung Hero”,"For the past year and a half, my daughter has ...","{'username': 'LeviK', 'num_reviews': 2, 'num_c...",July 2011,98979,0,2011-07-28,115879719,False
4,"{'service': 5.0, 'cleanliness': 5.0, 'overall'...","“Great Value for a King, Queen, and Princesses.”","Great Value for a King, Queen, and Princesses....","{'username': 'thomrho', 'num_cities': 13, 'num...",March 2007,112273,8,2007-03-21,7198417,False


In [9]:
train.columns

Index(['ratings', 'title', 'text', 'author', 'date_stayed', 'offering_id',
       'num_helpful_votes', 'date', 'id', 'via_mobile'],
      dtype='object')

In [10]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23999 entries, 0 to 23998
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   ratings            23999 non-null  object        
 1   title              23999 non-null  object        
 2   text               23999 non-null  object        
 3   author             23999 non-null  object        
 4   date_stayed        23050 non-null  object        
 5   offering_id        23999 non-null  int64         
 6   num_helpful_votes  23999 non-null  int64         
 7   date               23999 non-null  datetime64[ns]
 8   id                 23999 non-null  int64         
 9   via_mobile         23999 non-null  bool          
dtypes: bool(1), datetime64[ns](1), int64(3), object(5)
memory usage: 1.7+ MB


In [22]:
train['ratings'][0]

{'service': 1.0,
 'cleanliness': 2.0,
 'overall': 1.0,
 'value': 1.0,
 'location': 3.0,
 'sleep_quality': 2.0,
 'rooms': 1.0}

In [23]:
unique_cols = defaultdict(int)

for i in range(len(train)):
    for n in train['ratings'][i].keys():
        unique_cols[n] += 1

unique_cols

defaultdict(int,
            {'service': 21550,
             'cleanliness': 21545,
             'overall': 23999,
             'value': 21547,
             'location': 18983,
             'sleep_quality': 14185,
             'rooms': 20000,
             'check_in_front_desk': 2909,
             'business_service_(e_g_internet_access)': 1959})